https://github.com/jupyter/notebook/issues/2287

incase you run into an issue with running the code due to exceeding run rate (?)

In [1]:
# imports for database connection
import pandas as pd
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy import create_engine, func

# import these to view table column headers & rows
from sqlalchemy.orm import sessionmaker
from sqlalchemy import select

# Import the dependencies for analysis
%matplotlib inline

import numpy as np
import pandas as pd
import datetime as dt
from datetime import date
from datetime import timedelta
import matplotlib.pyplot as plt

In [2]:
# UPDATE PATH ONCE FINAL ANALYSIS FILE IS COMPILED
engine = create_engine("sqlite:///../../Resources/covid_db.db")

In [3]:
# reflect an existing database into a new model (creates base class for automap schema)
Base = automap_base()

In [4]:
# reflect the tables
Base.prepare(engine, reflect=True)

In [5]:
Base.classes.keys()

['covid_stats', 'government_regulation']

In [6]:
# save references to each table
covid_stats = Base.classes.covid_stats
government_regulation = Base.classes.government_regulation

In [7]:
# Create session (link) from Python to the database
session = sessionmaker(bind=engine)()

In [8]:
# perform a query to retrieve the data
results = session.query(government_regulation.primary_key_response,
                       government_regulation.Date_updated,
                       government_regulation.CountryName,
                       government_regulation.CountryCode,
                       government_regulation.StringencyIndex_updated,
                       government_regulation.percentage_change_stringency,
                       government_regulation.StringencyLegacyIndex_updated,
                       government_regulation.GovernmentResponseIndex_updated,
                       government_regulation.percentage_change_GovernmentResponse,
                       government_regulation.ContainmentHealthIndex_updated,
                       government_regulation.percentage_change_ContainmentHealth ,
                       government_regulation.EconomicSupportIndex_updated,
                       government_regulation.percentage_change_EconomicSupport
                       ).all()
    
# save the query results as a Pandas DataFrame and set the index
government_regulation = pd.DataFrame(results, columns=['primary_key_response',
                                                       'Date_updated',
                                                       'CountryName',
                                                       'CountryCode',
                                                       'StringencyIndex_updated',
                                                       'percentage_change_stringency',
                                                       'StringencyLegacyIndex_updated',
                                                       'GovernmentResponseIndex_updated',
                                                       'percentage_change_GovernmentResponse',
                                                       'ContainmentHealthIndex_updated',
                                                       'percentage_change_ContainmentHealth',
                                                       'EconomicSupportIndex_updated',
                                                       'percentage_change_EconomicSupport'
                                                      ])

In [9]:
# preview 
government_regulation

,primary_key_response,Date_updated,CountryName,CountryCode,StringencyIndex_updated,percentage_change_stringency,StringencyLegacyIndex_updated,GovernmentResponseIndex_updated,percentage_change_GovernmentResponse,ContainmentHealthIndex_updated,percentage_change_ContainmentHealth,EconomicSupportIndex_updated,percentage_change_EconomicSupport
0,ABW2020-01-01,2020-01-01,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
1,ABW2020-01-02,2020-01-02,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
2,ABW2020-01-03,2020-01-03,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
3,ABW2020-01-04,2020-01-04,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
4,ABW2020-01-05,2020-01-05,Aruba,ABW,0.00,0.0,0.0,0.00,0.0,0.00,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
43671,ZWE2020-08-27,2020-08-27,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0
43672,ZWE2020-08-28,2020-08-28,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0
43673,ZWE2020-08-29,2020-08-29,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0
43674,ZWE2020-08-30,2020-08-30,Zimbabwe,ZWE,80.56,0.0,88.1,66.03,0.0,73.48,0.0,25.0,0.0


# Exploratory Analysis

In [10]:
Aruba = pd.DataFrame(government_regulation.loc[government_regulation["CountryName"]=="Aruba"].reset_index()).drop(["index"],axis=1)

In [11]:
Aruba["StringencyIndex_updated"].max()

88.89

In [12]:
# Filter based on percentage changes greater than 10% in policy - CANADA
Aruba = Aruba.loc[(Aruba.StringencyIndex_updated >= 10) &
                   (Aruba.percentage_change_stringency > 0) &
                   (Aruba.StringencyIndex_updated <= Aruba["StringencyIndex_updated"].max())
                  ]
Aruba['Date_updated'] = pd.to_datetime(Aruba['Date_updated'])
Aruba.reset_index(inplace=True)

In [13]:
Aruba

,index,primary_key_response,Date_updated,CountryName,CountryCode,StringencyIndex_updated,percentage_change_stringency,StringencyLegacyIndex_updated,GovernmentResponseIndex_updated,percentage_change_GovernmentResponse,ContainmentHealthIndex_updated,percentage_change_ContainmentHealth,EconomicSupportIndex_updated,percentage_change_EconomicSupport
0,76,ABW2020-03-17,2020-03-17,Aruba,ABW,22.22,100.000000,28.57,17.95,74.951267,21.21,75.000000,0.0,0.0
1,77,ABW2020-03-18,2020-03-18,Aruba,ABW,33.33,50.000000,42.86,25.64,42.841226,30.30,42.857143,0.0,0.0
2,80,ABW2020-03-21,2020-03-21,Aruba,ABW,44.44,33.333333,60.71,37.18,26.076636,43.94,26.083214,0.0,0.0
3,88,ABW2020-03-29,2020-03-29,Aruba,ABW,85.19,91.696670,85.71,65.38,75.847230,77.27,75.853437,0.0,0.0
4,100,ABW2020-04-10,2020-04-10,Aruba,ABW,88.89,4.343233,85.71,81.41,3.246671,80.30,3.921315,87.5,0.0
5,102,ABW2020-04-12,2020-04-12,Aruba,ABW,88.89,4.343233,85.71,81.41,3.246671,80.30,3.921315,87.5,0.0
6,216,ABW2020-08-04,2020-08-04,Aruba,ABW,43.52,34.279543,55.95,48.72,18.742384,46.21,24.488147,62.5,0.0
7,219,ABW2020-08-07,2020-08-07,Aruba,ABW,47.22,8.501838,59.52,51.28,5.254516,49.24,6.557022,62.5,0.0
8,230,ABW2020-08-18,2020-08-18,Aruba,ABW,58.33,23.528166,73.81,58.97,14.996100,58.33,18.460601,62.5,0.0
9,238,ABW2020-08-26,2020-08-26,Aruba,ABW,62.04,6.360363,75.00,61.54,4.358148,61.36,5.194583,62.5,0.0


In [23]:
ramp_up_max = Aruba.StringencyIndex_updated.max()
ramp_up_end_date = Aruba.loc[Aruba.StringencyIndex_updated == ramp_up_max]
ramp_up_end_date = ramp_up_end_date.drop_duplicates(subset=['StringencyIndex_updated'], keep='first')
ramp_up_date = ramp_up_end_date['Date_updated']
index = ramp_up_date.index

In [24]:
index

Int64Index([4], dtype='int64')

In [25]:
Aruba.drop(Aruba.index[np.where(Aruba.index > index)[0]])

ValueError: Lengths must match to compare

In [ ]:
Aruba = Aruba.loc[Aruba['Date_updated'] < ramp_up_end ]

In [ ]:
Zimbabwe['Date_updated'] = pd.to_datetime(Zimbabwe['Date_updated'])

In [ ]:
number_of_days = Zimbabwe['Date_updated'].max() - Zimbabwe['Date_updated'].min()
number_of_days.days

In [ ]:
countries = government_regulation["CountryName"].unique()

In [ ]:
# List or processed iso dataframes
ramp_up = []

In [ ]:
# Loop through all countries and capture ramp up period
for location in countries:
    country = government_regulation.loc[government_regulation["CountryName"]==location].copy()
   
    country = country.loc[(country.StringencyIndex_updated >= 10) &
                   (country.percentage_change_stringency > 0) &
                   (country.StringencyIndex_updated <= country["StringencyIndex_updated"].max())
                  ]
    
    country['Date_updated'] = pd.to_datetime(country['Date_updated'])
    ramp_up.append(country)

ramp_up_df = pd.DataFrame()
for df in ramp_up:
    ramp_up_df = new_df.append(df).reset_index()

In [ ]:
ramp_up_df

In [ ]:
country

In [ ]:
# Set the x-axis to a date_updated and y-axis to percentage change for Canada
x_axis = new_canada_GT10["date_updated"]
y_axis = new_canada_GT10["percentage_change_stringency"]

In [ ]:
# Plot material percentage change for stringency index - CANADA
plt.plot(x_axis , y_axis)

plt.xticks(rotation=60)

plt.title("Material (+10% or <0%) Stringency Index Percentage Change - Canada")